In [22]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder



In [23]:
data_dir = '/home/panda/model_data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['None','push', 'pull', 'up', 'down', 'stop','rotate_clockwise','rotate_counterclockwise','rotate_up','rotate_down']
# Iterate over each subdirectory (class folder) in the data directory
for i,class_name in enumerate    (class_names):
    
    class_path = os.path.join(data_dir, class_names[i])
    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            # count+=1
            if file_name.endswith('.npy'):
                  
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                # Check if the shape of the recorded array matches the expected shape
                expected_shape = (210, )  # Adjust the shape according to your data
                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    count+=1
                    recorded_array=[]
                else:
                    continue
    print(f'data points in class {class_name} are {count}') 
    count=0         
# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Step 2: Preprocess the data

# Perform any required preprocessing steps, such as scaling or normalization
# Here, we will use StandardScaler to scale the data
print(data.shape)
print (labels.shape)
print(len(class_names))
print(count)

data points in class None are 2500
data points in class push are 2000
data points in class pull are 2000
data points in class up are 2000
data points in class down are 2000
data points in class stop are 4000
data points in class rotate_clockwise are 2000
data points in class rotate_counterclockwise are 2000
data points in class rotate_up are 1994
data points in class rotate_down are 2000
(22494, 210)
(22494,)
10
0


In [81]:
data_dir = '/home/rehan/catkin_ws/src/panda_research/hand_classifier/data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['push', 'pull', 'up', 'down', 'stop','rotate_cw','rotate_ccw']
class_counts = []  # Initialize a list to store the counts of valid samples per class

# Iterate over each subdirectory (class folder) in the data directory
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_dir, class_name)
    class_count = 0  # Initialize the count for the current class

    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)

            if file_name.endswith('.npy'):
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                expected_shape = (210,)  # Adjust the shape according to your data

                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    class_count += 1  # Increment the count for the current class

    class_counts.append(class_count)  # Store the count for the current class

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Print the counts for each class
for i, class_name in enumerate(class_names):
    print(f"{class_name}: {class_counts[i]}")

push: 2000
pull: 2000
up: 2000
down: 2000
stop: 2000
rotate_cw: 2000
rotate_ccw: 2000


In [24]:
# Split the data and labels into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [25]:
scaler = StandardScaler()
normalizer = MinMaxScaler(feature_range=(0, 1))

data_scaled = scaler.fit_transform(data)
data_normalized = normalizer.fit_transform(data_scaled)

train_data, val_data, train_labels, val_labels = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

print(train_data.shape)
print(val_data.shape)
print(train_labels.shape)
print(val_labels.shape)

(17995, 210)
(4499, 210)
(17995,)
(4499,)


In [26]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(210,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))


In [27]:

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [28]:
# Train the model
model.fit(train_data, train_labels, batch_size=32, epochs=100, validation_data=(val_data, val_labels))

Epoch 1/100
563/563 [==============================] - 4s 6ms/step - loss: 1.4080 - accuracy: 0.5044 - val_loss: 0.3564 - val_accuracy: 0.9442
Epoch 2/100
563/563 [==============================] - 3s 5ms/step - loss: 0.6083 - accuracy: 0.7878 - val_loss: 0.1462 - val_accuracy: 0.9820
Epoch 3/100
563/563 [==============================] - 3s 6ms/step - loss: 0.4436 - accuracy: 0.8433 - val_loss: 0.0693 - val_accuracy: 0.9916
Epoch 4/100
563/563 [==============================] - 3s 6ms/step - loss: 0.3467 - accuracy: 0.8796 - val_loss: 0.0461 - val_accuracy: 0.9931
Epoch 5/100
563/563 [==============================] - 3s 6ms/step - loss: 0.2962 - accuracy: 0.8980 - val_loss: 0.0353 - val_accuracy: 0.9973
Epoch 6/100
563/563 [==============================] - 3s 6ms/step - loss: 0.2609 - accuracy: 0.9128 - val_loss: 0.0370 - val_accuracy: 0.9924
Epoch 7/100
563/563 [==============================] - 3s 6ms/step - loss: 0.2347 - accuracy: 0.9193 - val_loss: 0.0281 - val_accuracy: 0.9956

In [29]:
# Get the 9predicted labels from the model
predicted_labels = model.predict(val_data)
predictions = np.around(model.predict(val_data))
# print(predictions)
# Decode the predicted labels
print((predicted_labels.shape))
for i,prediction in enumerate (predictions):
    # print(np.argmax(predictions[i][0]))
    label=np.argmax(predictions)
    print(class_names[label])
# print(prediction_labels[0][np.argmax(prediction)])
print(class_names)

141/141 [==============================] - 0s 2ms/step
(4499, 10)
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rotate_up
rota

In [21]:
import pickle
# Save the model
model.save('/home/panda/model_data/model2/keras_model.h5')

# Save the labels to a text file
with open('/home/panda/model_data/model2/labels.txt', 'w') as f:
    for label in class_names:
        f.write(label + '\n')
# Saving the scaler and normalizer objects
with open('/home/panda/model_data/model2/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

with open('/home/panda/model_data/model2/normalizer.pkl', 'wb') as normalizer_file:
    pickle.dump(normalizer, normalizer_file)